# <font color ='green' > Common


In [11]:
from sklearn.metrics import fbeta_score, make_scorer

from imblearn.over_sampling import SMOTE

from imblearn.combine import SMOTETomek # over sampling method 1

## RandomOverSampler to handle imbalanced data
from imblearn.over_sampling import RandomOverSampler # over sampling method 2


#Shuffle the dataframe
# from sklearn.utils import shuffle # df1 = shuffle(df).reset_index(drop = True)

from imblearn.under_sampling import NearMiss
from pandas.api.types import is_numeric_dtype

from pickle import dump, load
from joblib import dump, load

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


from sklearn.base import TransformerMixin, BaseEstimator
from collections import Counter , defaultdict


pd.set_option('display.max_columns', None)


# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc

%matplotlib inline
rc["figure.figsize"] = 10,6


#Data Science and ML
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.feature_selection import RFE #Recursive Feature Selection
import statsmodels.api as sm 

#preprocessing
from sklearn.preprocessing import StandardScaler, normalize, LabelEncoder

from sklearn.pipeline import Pipeline


## <font color ='green' >1) Dataset Understand, check & get detail, How much Missing values present

In [ ]:
def understanding_dataset(dataset):
    print(f"Shape: {dataset.shape}")
    print(f"Total Missing Value in Dataset: {dataset.isna().sum().sum()}")
    
    for column in dataset.columns:
        print(f"===============Column: {column} ==============")
        print(f"Number of unique values: {dataset[column].nunique()}")
        print(f"Max: {dataset[column].max()}")
        print(f"Min: {dataset[column].min()}")
        
        if(dataset[column].isna().any()):
            print(f"Missing Value: {round((data[[column]].isna().sum() / len(data) ) * 100 , 2)}")
    
        print("\n")

def get_percentage_miss_value(dataset):
    higher_miss_value_column = []
    miss_threshold_value = 50
    
    for i in dataset.columns:
        if dataset[i].isna().sum() > 1: 
            perectange_val = (dataset[i].isna().sum() / len(dataset)) * 100
            print("Column-> " , i, ", total no of missing value : ",dataset[i].isna().sum() , " & :         ", round(perectange_val,2) ," %")
                
            if(perectange_val > miss_threshold_value):
                higher_miss_value_column.append(i)
            
    print("\n\n\n")
    
    if higher_miss_value_column:
        print("Higher Missing values in Columns Name : ", higher_miss_value_column)
    else:
        print("There are no Higher Column Missing values in Dataset")
        
            
def check_null(dataframe, percentage=True):
    if percentage==True:
        return round(dataframe.isnull().mean()[dataframe.isnull().mean()>0].apply(lambda x: x*100),3)
    else:
        return dataframe.isnull().sum()[dataframe.isnull().sum()>0]
    
check_null(train_data, percentage=False)

def remove_digit_numerical(dataset):
    for i in dataset.columns:
        dataset[i] = pd.to_numeric(dataset[i].str.replace(r"[^\d]", ""))
    return dataset


In [ ]:
def check_cloumn_details_type_categorical(dataset):
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            
            print("*"*100)
            print("\n")

def check_cloumn_details_type_numberical(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "int"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
            
def check_cloumn_details_type_float(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "float"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            

In [ ]:
def get_categorical_col(dataset): #testing has been pending
    return dataset.select_dtypes(include=['object']) #get all categorical columns only

def get_numercial_data_types_col_name(dataset):
    return list(dataset.select_dtypes(include =  ["int" , "float"] ).columns)

def convert_numerical_data_type(dataset):
    col_list = list(dataset.select_dtypes(include =  ["int" , "float"] ).columns)
    for numberical_colname in col_list:
        dataset[numberical_colname] = pd.to_numeric(dataset[numberical_colname])  
    return dataset
    
def get_column_integer_float_list(dataset):
    numberical_int_columns = []
    for i in dataset.columns:
        if dataset[i].dtype == "int":
            numberical_int_columns.append(i)
        
    numberical_float_columns = []
    for i in dataset.columns:
        if dataset[i].dtype == "float":
            numberical_float_columns.append(i)
            
    return numberical_int_columns, numberical_float_columns


# <font color ='green' >2) Preprocessing

### <font color ='blue' >2.1)  Impute for missing value(Numberical : mean, categorical :mode(0))

In [ ]:
class DataFrameImputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
df6 = DataFrameImputer().fit_transform(df5)

### <font color ='red' >2.2)  Different Types of Encoding and convert categorial to numerical

In [ ]:
def convet_categorical_numerical_use_get_dummy(dataset):
    df2_categorical = dataset.select_dtypes(include =  ["object"] )
    df_dummies = pd.get_dummies(df2_categorical,drop_first=True)
    
    # drop categorical variables 
    dataset = dataset.drop(columns=list(df2_categorical.columns), axis=1)
    
    # concat dummy variables with X
    dataset = pd.concat([dataset, df_dummies], axis=1)
    
    return dataset


# We one-hot encode the categorical features
def one_hot_encoding(rides):
    dummy_fields = ['season', 'weather', 'month', 'hour', 'weekday']
    for each in dummy_fields:
        dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
        rides = pd.concat([rides, dummies], axis=1)
    return rides


#Label encoding
def convert_to_numerical_label_encoding(dataset):
    enc = LabelEncoder()
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            dataset[i] = enc.fit_transform(dataset[i])
            
    return dataset

#function for converting categoric to num codes
#Changing Categorical colum values to numeric codes¶

def cat_to_num(dataset):
    for i in range(0, dataset.shape[1]):
        #print(i)
        if(dataset.iloc[:,i].dtypes == 'object'):
            dataset.iloc[:,i] = pd.Categorical(dataset.iloc[:,i])
            dataset.iloc[:,i] = dataset.iloc[:,i].cat.codes
            dataset.iloc[:,i] = dataset.iloc[:,i].astype('object')
    return dataset


train = cat_to_num(train)
test = cat_to_num(test)

gender_encoder = OneHotEncoder(dtype=np.uint8)
gender = gender_encoder.fit_transform(df["Gender"].values.reshape(-1,1)).todense()

X = np.array(np.hstack((X,cabs,life,dest,gender)))
print (np.array(X),y)

In [ ]:
#seeking only the numeric features from the data or selecting only categorical column
numeric_features = df2.select_dtypes(include = [np.number] or ["object"] or [np.float])

df['ExterQual'] = df.ExterQual.map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

### <font color ='red' > 2.3) Scaling & Transform & Drop Outlier

In [ ]:
def standardscaler_preprocessing(dataset_train, dataset_test, num_col):
    scaler = StandardScaler()
   
    dataset_train[num_col] = scaler.fit_transform(dataset_train[num_col])

    dataset_test[num_col] = scaler.transform(dataset_test[num_col])
    
    return dataset_train, dataset_test

def remove_outlier(dataframe):
    low = .05
    high = .95
    quant_df = dataframe.quantile([low, high])
    for name in list(dataframe.columns):
        if is_numeric_dtype(dataframe[name]):
            dataframe = dataframe[(dataframe[name] > quant_df.loc[low, name]) & (dataframe[name] < quant_df.loc[high, name])]
    return dataframe


def drop_outliers(dataset, columns_name):
    for chk in columns_name:
        
#         plt.boxplot(dataset[chk]) # please with graph & it is optional
        
        Q1 = dataset[chk].quantile(.25)
        Q3 = dataset[chk].quantile(.75)
        IQR = Q3-Q1
        dataset =  dataset[(dataset[chk] >= (Q1-(1.5*IQR))) & (dataset[chk] <= (Q3+(1.5*IQR)))] 
    return dataset


## <font color ='red' > 2.3.b) Standard Scaler for Both C&R, but use only 1 or more or list integer cols only

In [ ]:
scale_columns_int_list = ["int_col_1","int_col_2", "int_col_3","int_col_4"]

scaler = StandardScaler()
transformer = ColumnTransformer([ ("scaler", scaler, scale_columns_int_list)])

transform_X = transformer.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(transform_X,y,test_size=0.2)

### <font color ='green' > 2.4) VIF : variance inflation feature

In [ ]:
#Using VIF for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.Series([variance_inflation_factor(X_train.values, j) for j in range( X_train.shape[1])],index=X_train.columns)

#or

def calculate_vif(X_train_val): # x : 
    thresh=10.0
    output= X_train_val
    k=X_train_val.shape[1]
    
    vif=[variance_inflation_factor(X_train_val.values,j) for j in range(X_train_val.shape[1])]
    for i in range(1,k):#till k cz atleast one column would be kept even if its greater than 5
        print('iteration no',i)
        print(vif)
        a=np.argmax(vif)
        print('max vif is for variable no',a)
        print(" ")
        if vif[a]<=thresh:
            break
        else:
            output=output.drop(output.columns[a],axis=1)
            vif=[variance_inflation_factor(output.values,j) for j in range(output.shape[1])]
    return output

X_vif=calculate_vif(X_train)

X_train_dup=X_vif
X_train_dup.columns

X_test= X_test[X_train_dup.columns]

# <font color ='red' >4) Imabalnced Dataset : Over Sampling(Majoruity)

In [127]:
def majority_imbalanced_dataset(dataset, target_col):
    feature_columns = dataset.columns.tolist()
    feature_columns = [c for c in feature_columns if c not in [target_col]]

    X2_new = dataset[feature_columns]
    Y2_new = dataset[target_col]

    os =  RandomOverSampler(random_state=35)
    X_feature_variables , y_output = os.fit_sample(X2_new, Y2_new)
    
    X_feature_variables[target_col] = y_output
    
    X_feature_variables = X_feature_variables.sample(frac = 1).reset_index(drop = True)
    
    return X_feature_variables

Counter(df2.target)
df3 = majority_imbalanced_dataset(df2.copy()   , "target")
Counter(df3.target)


# df4 = majority_imbalanced_dataset(df3.copy(), "Response")
# print(len(df4[df4["Response"] == 0]), len(df4[df4["Response"] == 1]))
# df4.head()

In [ ]:

dataset['Origin'] = dataset['Origin'].map(lambda x: {1: 'USA', 2: 'Europe', 3: 'Japan'}.get(x))


### Split the data into train and test

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)


sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

sns.pairplot(dataset, diag_kind="kde")

In [ ]:
dataset.describe().transpose()